In [ ]:
pip install chromadb sentence_transformers beautifulsoup4 requests

In [ ]:
import requests
from bs4 import BeautifulSoup # we use BeatifulSoup for webscraping
term = "202610" #2025-2026 course catalog
base_url = "https://courses.rice.edu"
url_catalog = base_url + "/admweb/!SWKSCAT.cat?p_action=CATALIST&p_term=" + term # scrape data from courses.rice.edu

# Get the HTML code of the page
resp = requests.get(url_schedule)
html_source = resp.text

soup = BeautifulSoup(html_source, "html.parser") # our scraper

courses = []

for row in soup.select("tr"): # iterate through each table row
    cells = row.find_all("td") # get all table data  cells in current table row
    if len(cells) > 4 and cells[0].find("a"): # skip rows that don't have enough cells (header or blank rows)
        courses.append({
            "course": cells[0].text.strip(),
            "title": cells[1].text.strip(),
            "distribution_group": cells[2].text.strip(),
            "diversity_credit": cells[3].text.strip() != "",
            "credit_hours": cells[4].text.strip(),
        })

        # Get the url
        link_tag = cells[0].find("a")
        relative_url = link_tag['href']
        full_url = base_url+relative_url

        #Here, scrape the data from sub-page.

        # Recommend using "for row in soup.select("tr")[:10]" for the loop, since
        # scraping sub-pages of a long list of courses may be a slow process, and unless
        # you are sure your code correctly scrapes sub-pages, run it only on a small sublist of courses

for course in courses:
    print(course)

In [ ]:
from sentence_transformers import SentenceTransformer

# Get the model we will be using for embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize fields of our database
documents = []
metadatas = []
ids = []
id = 0

# Iterate over our scraped courses
for course in schedule_data:

  # Create a single string that encapsulates course title, code, and description - maximum context for embedding
  doc_text = f"Course: {course['title']} : {course['course']}. Description: {course['description']}"

  documents.append(doc_text)
  metadatas.append(course)
  ids.append(str(id))
  id += 1

embeddings = model.encode(documents, show_progress_bar=True)

In [ ]:
import chromadb


'''
You can use chromadb's embedding with following changes in code:

from chromadb.utils import embedding_functions

embedding = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

db_path = 'rice_courses_db_chromadb_embedding'
client = chromadb.PersistentClient(path = db_path)

collection = client.create_collection(
    name = "rice_courses_chromadb_embedding",
    embedding_function=embedding
    )
collection.add(
    documents = documents,
    metadatas = metadatas,
    ids = ids
    )
'''
# Create out database

db_path = 'rice_courses_db'
client = chromadb.PersistentClient(path = db_path)
collection = client.get_or_create_collection(
    name = "rice_courses",
    )

# Add data to the database
collection.add(
    documents = documents,
    metadatas = metadatas,
    ids = ids
    )

In [ ]:
results = collection.query(
    query_texts = ["Class about African history"],
    n_results = 1 # by default returns 10 closest results
)
print(results["documents"])
print(results['distances'])